## Часть 1. Генерация информации

In [ ]:
import csv
from faker import Faker
import random
fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])

    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)

        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")


Сгенерировано 100000 записей и сохранено в web_server_logs.csv


## Часть 2. Анализ информации

In [71]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, count, to_date, sum


spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

path = r'/content/web_server_logs.csv'
df_logs = spark.read.csv(path, header=True, inferSchema=True)

#### 1. Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP. Формат вывода, как на скрине ниже.

In [75]:
print('Top 10 active IP adresses:')
df_logs.groupBy('ip').agg(count("ip").alias("request_count"))\
.orderBy(desc('request_count')).show(10)

Top 10 active IP adresses
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
| 219.82.251.112|            2|
| 201.182.32.102|            1|
|   57.48.146.32|            1|
|  22.229.67.159|            1|
|143.129.104.167|            1|
|     31.43.1.71|            1|
| 156.72.245.135|            1|
| 97.219.102.197|            1|
|   83.28.80.199|            1|
| 205.112.110.22|            1|
+---------------+-------------+
only showing top 10 rows



#### 2. Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.



In [76]:
print('Request count by HTTP method:')
df_logs.groupBy('method').agg(count("method").alias("method_count")).show()

Request count by HTTP method
+------+------------+
|method|method_count|
+------+------------+
|  POST|       24841|
|DELETE|       25029|
|   PUT|       25249|
|   GET|       24881|
+------+------------+



#### 3. Профильтруйте и посчитайте количество запросов с кодом ответа 404.


In [79]:
 print(f'Number of 404 respose codes: {df_logs.filter(df_logs.response_code == 404).count()}')

Number of 404 respose codes: 25090


#### 4. Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

In [80]:
df_logs = df_logs.withColumn('date', to_date(df_logs['timestamp']).alias('date'))
print('Total response size by day:')
df_logs.groupBy('date').agg(sum('response_size').alias('total_response_size'))\
.orderBy('date').show()

Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2024-01-01|            1725503|
|2024-01-02|            1602519|
|2024-01-03|            1496979|
|2024-01-04|            1659664|
|2024-01-05|            1526194|
|2024-01-06|            1657838|
|2024-01-07|            1709296|
|2024-01-08|            1638062|
|2024-01-09|            1587850|
|2024-01-10|            1512633|
|2024-01-11|            1647220|
|2024-01-12|            1641089|
|2024-01-13|            1537883|
|2024-01-14|            1577268|
|2024-01-15|            1666068|
|2024-01-16|            1530095|
|2024-01-17|            1766293|
|2024-01-18|            1506113|
|2024-01-19|            1518652|
|2024-01-20|            1619930|
+----------+-------------------+
only showing top 20 rows

